costruito una pipeline data-driven che, a partire dallo storico del Draft NBA, pulisce il dataset, ingegnerizza le feature (es. PickBand), addestra un modello per stimare l’Expected_WS di ogni giocatore e ti permette di calcolare il Delta (WS reale – atteso), così da misurare in modo quantitativo steal e bust del draft.

In [2]:
import os
import pandas as pd
import numpy as np

# ============================
# 1. Config & caricamento dati
# ============================

# Path al dataset pulito della PARTE 1
# Cambia il path se diverso nel tuo progetto
DATA_PATH = "../data/drafted_cleaned.csv"

df = pd.read_csv(DATA_PATH)

print("Shape dataset:", df.shape)
print(df.columns)

# (Opzionale) tieni solo i giocatori con almeno X minuti / partite, se hai queste colonne
# if 'MP' in df.columns:
#     df = df[df['MP'] >= 500]

# ===================================
# 2. Funzioni helper per descrizioni
# ===================================

def safe_get(row, col, default=None):
    """Restituisce row[col] se esiste e non è NaN, altrimenti default."""
    if col in row and not pd.isna(row[col]):
        return row[col]
    return default

def format_percent(value, decimals=1):
    """Converte 0.385 in '38.5%' oppure 38.5 in '38.5%' a seconda di come è salvato."""
    if value is None:
        return None
    try:
        v = float(value)
    except (TypeError, ValueError):
        return None
    if v <= 1:   # tipo 0.385
        v = v * 100
    return f"{v:.{decimals}f}%"

def bucketize_3p(pct):
    """Ritorna un aggettivo qualitativo per il tiro da 3."""
    if pct is None:
        return None
    try:
        v = float(pct)
    except (TypeError, ValueError):
        return None
    if v <= 1:
        v = v * 100
    if v >= 40:
        return "ottimo tiratore da 3 punti"
    elif v >= 35:
        return "buon tiratore da 3 punti"
    elif v >= 30:
        return "tiratore da 3 punti discreto"
    else:
        return "tiratore da 3 punti poco affidabile"

def bucketize_ft(pct):
    """Aggettivo per i tiri liberi."""
    if pct is None:
        return None
    try:
        v = float(pct)
    except (TypeError, ValueError):
        return None
    if v <= 1:
        v = v * 100
    if v >= 85:
        return "eccellente tiratore di tiri liberi"
    elif v >= 75:
        return "buon tiratore di tiri liberi"
    else:
        return "tiratore di liberi migliorabile"

def bucketize_reb(reb):
    """Aggettivo per rimbalzi totali."""
    if reb is None:
        return None
    try:
        v = float(reb)
    except (TypeError, ValueError):
        return None
    if v >= 9:
        return "forte rimbalzista"
    elif v >= 6:
        return "buon rimbalzista"
    elif v >= 4:
        return "rimbalzista discreto"
    else:
        return "rimbalzista non di impatto"

def bucketize_ast(ast):
    """Aggettivo per assist."""
    if ast is None:
        return None
    try:
        v = float(ast)
    except (TypeError, ValueError):
        return None
    if v >= 7:
        return "ottimo playmaker e passatore"
    elif v >= 4:
        return "buon passatore"
    else:
        return "non particolarmente orientato agli assist"

def bucketize_def(ws_def):
    """Aggettivo per difesa basato su Defensive WS o simile."""
    if ws_def is None:
        return None
    try:
        v = float(ws_def)
    except (TypeError, ValueError):
        return None
    if v >= 3:
        return "difensore di alto livello"
    elif v >= 1.5:
        return "buon difensore"
    else:
        return "difensore nella media"

# =======================================
# 3. Funzione principale: riga -> testo
# =======================================

def build_player_text(row):
    """
    Trasforma un giocatore in una descrizione testuale in italiano.
    Qui puoi aggiungere / modificare logica a piacere.
    """

    pieces = []

    name = safe_get(row, "Player")
    if name:
        pieces.append(f"Giocatore: {name}.")

    # Ruolo / posizione
    pos = safe_get(row, "Pos") or safe_get(row, "Position")
    if pos:
        pieces.append(f"Ruolo: {pos}.")

    # Info su team/college/paese se le hai
    college = safe_get(row, "College")
    if college:
        pieces.append(f"Provenienza: {college}.")

    # Draft info
    draft_year = safe_get(row, "DraftYear")
    pick = safe_get(row, "Pick")
    pick_band = safe_get(row, "PickBand")
    if draft_year:
        if pick:
            pieces.append(f"Scelto al Draft NBA {int(draft_year)} con la pick {int(pick)}.")
        else:
            pieces.append(f"Giocatore del Draft NBA {int(draft_year)}.")
    if pick_band:
        pieces.append(f"Fascia di scelta: {pick_band}.")

    # Tiro da 3
    p3 = safe_get(row, "3P%") or safe_get(row, "ThreeP")
    if p3 is not None:
        p3_str = format_percent(p3)
        bucket_3p = bucketize_3p(p3)
        if p3_str and bucket_3p:
            pieces.append(f"Tiro da 3: {p3_str}, {bucket_3p}.")

    # Tiri liberi
    ft = safe_get(row, "FT%") or safe_get(row, "FT")
    if ft is not None:
        ft_str = format_percent(ft)
        bucket_ft = bucketize_ft(ft)
        if ft_str and bucket_ft:
            pieces.append(f"Tiri liberi: {ft_str}, {bucket_ft}.")

    # Rimbalzi
    reb = safe_get(row, "TRB") or safe_get(row, "REB") or safe_get(row, "TRB_per_game")
    bucket_reb = bucketize_reb(reb)
    if reb is not None and bucket_reb:
        pieces.append(f"Rimbalzi: {reb} a partita, {bucket_reb}.")
    
    # Rimbalzi offensivi
    orb = safe_get(row, "ORB")
    if orb is not None:
        try:
            orb_v = float(orb)
            if orb_v >= 3:
                pieces.append(f"Molto forte a rimbalzo offensivo ({orb_v} a partita).")
            elif orb_v >= 1.5:
                pieces.append(f"Buon rimbalzista offensivo ({orb_v} a partita).")
        except:
            pass

    # Assist
    ast = safe_get(row, "AST")
    bucket_ast = bucketize_ast(ast)
    if ast is not None and bucket_ast:
        pieces.append(f"Assist: {ast} a partita, {bucket_ast}.")
    
    # Stato
    status_it = row.get("Status_it", None)
    if status_it:
        parts.append(f"Stato: {status_it}.")

    # Difesa: Defensive WS o STL/BLK
    dws = safe_get(row, "DWS") or safe_get(row, "Def_WS")
    bucket_d = bucketize_def(dws)
    if bucket_d:
        pieces.append(bucket_d + ".")
    else:
        stl = safe_get(row, "STL")
        blk = safe_get(row, "BLK")
        try:
            stl_v = float(stl) if stl is not None else 0.0
            blk_v = float(blk) if blk is not None else 0.0
            if stl_v >= 1.5 or blk_v >= 1.5:
                pieces.append("Profilo difensivo interessante con buone statistiche di recuperi e stoppate.")
        except:
            pass

    # Punti per partita se disponibili
    pts = safe_get(row, "PTS") or safe_get(row, "PTS_per_game")
    if pts is not None:
        pieces.append(f"Segna in media {pts} punti a partita.")

    # Valutazione globale WS e Delta, dalla parte 1
    ws = safe_get(row, "WS")
    expected_ws = safe_get(row, "Expected_WS")
    delta = safe_get(row, "Delta")
    if ws is not None:
        pieces.append(f"Win Shares in carriera: {ws}.")
    if delta is not None:
        try:
            d = float(delta)
            if d > 20:
                pieces.append("Il rendimento in carriera è molto superiore alle attese rispetto alla sua posizione al draft.")
            elif d > 5:
                pieces.append("Il rendimento in carriera è superiore alle attese rispetto alla sua posizione al draft.")
            elif d < -10:
                pieces.append("Il rendimento in carriera è inferiore alle attese rispetto alla sua posizione al draft.")
        except:
            pass

    # Un'unica stringa finale
    text = " ".join(str(p) for p in pieces if p)
    return text

# ======================================
# 4. Applicazione al DataFrame & salvat.
# ======================================

df["text_profile"] = df.apply(build_player_text, axis=1)

# Dai un'occhiata ai primi esempi
for i in range(5):
    print("==== Giocatore", i, "====")
    print(df.loc[i, "text_profile"])
    print()

# Costruisci il corpus finale (puoi aggiungere altre colonne che ti servono)
corpus_cols = []
for col in ["Player", "DraftYear", "Pick", "PickBand", "Pos", "text_profile", "Status"]:
    if col in df.columns:
        corpus_cols.append(col)

corpus_df = df[corpus_cols].copy()

# Aggiungi un id univoco se non ce l'hai già
corpus_df["player_id"] = corpus_df.index.astype(int)

# Crea la cartella di output se non esiste
os.makedirs("../data/processed", exist_ok=True)

# Salva in JSON (comodo per IR)
corpus_json_path = "../data/processed/nba_players_corpus.json"
corpus_df.to_json(corpus_json_path, orient="records", force_ascii=False, indent=2)

# (Opzionale) Salva anche in CSV
corpus_csv_path = "../data/processed/nba_players_corpus.csv"
corpus_df.to_csv(corpus_csv_path, index=False)

print("Corpus salvato in:")
print(" -", corpus_json_path)
print(" -", corpus_csv_path)


Shape dataset: (8323, 35)
Index(['DraftYear', 'Pick', 'Tm', 'Player', 'College', 'Seasons', 'Games',
       'MP', 'PTS', 'TRB', 'AST', 'FG%', '3P%', 'FT%', 'MP.1', 'PTS.1',
       'TRB.1', 'AST.1', 'WS', 'WS/48', 'BPM', 'VORP', 'Forfeited', 'PickBand',
       'Debut', 'WS_available', 'WS/48_available', 'BPM_available',
       'VORP_available', 'MP_available', 'TRB_available', 'FG%_available',
       'FT%_available', 'MP.1_available', 'TRB.1_available'],
      dtype='object')
==== Giocatore 0 ====
Giocatore: Clifton McNeely. Provenienza: Texas Wesleyan University. Scelto al Draft NBA 1947 con la pick 1. Fascia di scelta: Top10. Tiro da 3: -100.0%, tiratore da 3 punti poco affidabile. Assist: 0.0 a partita, non particolarmente orientato agli assist. Win Shares in carriera: 0.0.

==== Giocatore 1 ====
Giocatore: Glen Selbo. Provenienza: Wisconsin. Scelto al Draft NBA 1947 con la pick 2. Fascia di scelta: Top10. Tiro da 3: -100.0%, tiratore da 3 punti poco affidabile. Tiri liberi: 75.9%, b

Carica il tuo dataset pulito della parte 1.

Per ogni giocatore costruisce una descrizione testuale in italiano con:

ruolo, provenienza, info draft (anno, pick, fascia),

tiro da 3 e liberi con aggettivi (ottimo/buono/etc.),

rimbalzi, rimbalzi offensivi, assist,

difesa (DWS, STL, BLK),

punti a partita,

WS, Expected_WS, Delta (se presenti).

Salva un corpus in:

nba_players_corpus.json

nba_players_corpus.csv